In [344]:
# Importing libraries neeeded for the project
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import mysql.connector
import pycountry
import numpy as np
import plotly.express as px


## Connexion à la base pour Wamp

In [345]:
# Connexion à la BD pour Wamp 
host = 'localhost'  
utilisateur = 'root'
mot_de_passe = '' 
base_de_donnees = 'ecotourisme'

# Connexion
connexion = mysql.connector.connect(
    host=host,
    user=utilisateur,
    password=mot_de_passe,
    database=base_de_donnees
)

curseur = connexion.cursor()

if connexion.is_connected():
    print("Connexion à la base de données réussie.")
else:
    print("Échec de la connexion à la base de données.")

Connexion à la base de données réussie.


## Connexion à la base avec Mamp

In [346]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

# import mysql.connector

# config = {
#   'user': 'root',
#   'password': 'root',
#   'host': '127.0.0.1',
#   'port': 8889,
#   'database': 'ecotourisme',
#   'raise_on_warnings': True
# }

# cnx = mysql.connector.connect(**config)

# curseur = cnx.cursor(dictionary=True)

## Traitement des données

In [347]:


curseur.execute('''
    SELECT 
        p.nom as pays, 
        s.id_pays, 
        s.annee, 
        s.gpi, 
        t.arriveesTotal as arriveesTotal, 
        e.pibParHab, 
        eco.GES_hab,
        eco.elecRenew,
        d.idh as idh
    FROM 
        surete as s, 
        tourisme as t, 
        economie as e, 
        ecologie as eco, 
        pays as p, 
        idh as d
    WHERE 
        p.id = s.id_pays 
        AND s.id_pays = t.id_pays 
        AND t.id_pays = e.id_pays 
        AND e.id_pays = eco.id_pays 
        AND d.id_pays = eco.id_pays
        AND s.annee = t.annee 
        AND t.annee = e.annee 
        AND e.annee = eco.annee 
        AND d.annee = eco.annee
        AND e.annee=2020;
''')

dataset = curseur.fetchall()

# Création de 4 dataframes pour chaque score 

df = pd.DataFrame(dataset, columns=['pays', 'id_pays', 'annee', 'gpi', 'arriveesTotal', 'pibParHab', 'GES_hab', 'elecRenew', 'idh'])
df_ecol = df[['pays', 'id_pays', 'annee', 'GES_hab', 'elecRenew']].copy()
df_eco = df[['pays', 'id_pays', 'annee', 'arriveesTotal','pibParHab','idh']].copy()
df_decouverte = df[['pays', 'id_pays', 'annee', 'arriveesTotal', 'GES_hab', 'idh']].copy()

# Fonction pour convertir ISO-2 en ISO-3
def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except AttributeError:
        return np.nan


## Score global ou score touristique

In [348]:
 
# Normalisation des indicateurs
df['arriveesTotal'] = df['arriveesTotal'] / df['arriveesTotal'].max()
df['gpi'] = df['gpi'].min() / df['gpi']
df['pibParHab'] = df['pibParHab'] / df['pibParHab'].max()
df['GES_hab'] = df['GES_hab'].min() / df['GES_hab']
df['elecRenew'] = df['elecRenew'] / df['elecRenew'].max()
df['idh'] = df['idh'] / df['idh'].max()

# Les poids pour chaque variable
poids = {
    'arriveesTotal': 2,
    'gpi': 4,
    'pibParHab': 2,
    'GES_hab': 6, 
    'elecRenew': 3,
    'idh' : 4
}

# Calcul du score touristique 
df['score_touristique'] = df.apply(lambda row: sum(float(row[var]) * poids[var] for var in poids) / sum(poids.values()), axis=1)

df.head()


max_score = df['score_touristique'].max()
min_score = df['score_touristique'].min()

variance = df['score_touristique'].std()
esperance = df['score_touristique'].mean()


# création des intervalles pour chaque label du score
bins = [min_score, 0.26, 0.34, 0.42, 0.5, max_score]

print("max : ",max_score, "min : ",min_score, "variance: ",variance, "esperance: ",esperance)


labels = ['E', 'D', 'C', 'B', 'A']

# Créer une nouvelle colonne score
df['label_touristique'] = pd.cut(df['score_touristique'], bins=bins, labels=labels)

df = df.sort_values(by='score_touristique', ascending=False)

# suppression des valeurs NaN
df.dropna(inplace=True)


# Visualisation des scores

# Création d'un histogramme interactif avec Plotly
fig = px.histogram(df, x='score_touristique', nbins=5, color_discrete_sequence=['#636EFA'], opacity=0.7)

# Ajout d'effets de relief
fig.update_traces(marker_line_color='white', marker_line_width=1, opacity=0.7)

# Ajout de lignes verticales pour séparer les catégories de scores
for i in range(1,len(bins)):  
    fig.add_vline(x=bins[i], line_dash='dash', line_color='gray', line_width=1)
fig.add_vline(x=max(df['score_touristique']), line_dash='dash', line_color='gray', line_width=1)

# Ajout de légendes pour les catégories de scores
for i, label in enumerate(labels):
    fig.add_annotation(x=bins[i] + (bins[i+1] - bins[i]) / 2, y=20, text=label, font=dict(color='black', size=12), showarrow=False)

# Mise en forme du graphique
fig.update_layout(
    title='Distribution du Score Touristique',
    xaxis_title='Score Touristique',
    yaxis_title='Nombre d\'observations',
    bargap=0.05,
    showlegend=False
)

# Affichage du graphique interactif
fig.show()

# Appliquer la fonction à la colonne 'id_pays'
df['id_pays'] = df['id_pays'].apply(iso2_to_iso3)

# Créer une carte du monde
fig = px.choropleth(df, locations='id_pays', color='label_touristique', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()


# 99 score sur 196 enregistré 

max :  0.5965868227056257 min :  0.1810022074886945 variance:  0.07938479582918744 esperance:  0.3883544190848649


## Score écologique

In [349]:

# Normalisation des indicateurs par la méthode min-max
df_ecol['GES_hab'] = df_ecol['GES_hab'].min() / df_ecol['GES_hab']
df_ecol['elecRenew'] = df_ecol['elecRenew'] / df_ecol['elecRenew'].max()

# Définir les poids pour chaque variable
poids_ecol = {
    'GES_hab': 3, 
    'elecRenew': 2
}

# Calculer le score écologique 
df_ecol['score_ecologique'] = df_ecol.apply(lambda row: sum(float(row[var]) * poids_ecol[var] for var in poids_ecol) / sum(poids_ecol.values()), axis=1)


df_ecol.head()

max_score = df_ecol['score_ecologique'].max()
min_score = df_ecol['score_ecologique'].min()

variance = df_ecol['score_ecologique'].std()
esperance = df_ecol['score_ecologique'].mean()

# Les limites des scores/labels
bins = [min_score, 0.17, 0.34, 0.51, 0.68, max_score]

print("max : ",max_score, "min : ",min_score, "variance: ",variance, "esperance: ",esperance)

# Les labels
labels = ['E', 'D', 'C', 'B', 'A']

# Créer une nouvelle colonne score
df_ecol['label_ecol'] = pd.cut(df_ecol['score_ecologique'], bins=bins, labels=labels)

df_ecol = df_ecol.sort_values(by='score_ecologique', ascending=False)

df_ecol.head()

df_ecol.dropna(inplace=True)


# Création d'un histogramme interactif avec Plotly
fig = px.histogram(df_ecol, x='score_ecologique', nbins=5, color_discrete_sequence=['#F5F5DC'], opacity=0.7)

# Ajout d'effets de relief
fig.update_traces(marker_line_color='white', marker_line_width=1, opacity=0.7)

# Ajout de lignes verticales pour séparer les catégories de scores
for i in range(1,len(bins)):  
    fig.add_vline(x=bins[i], line_dash='dash', line_color='gray', line_width=1)
fig.add_vline(x=max(df_ecol['score_ecologique']), line_dash='dash', line_color='gray', line_width=1)

# Ajout de légendes pour les catégories de scores
for i, label in enumerate(labels):
    fig.add_annotation(x=bins[i] + (bins[i+1] - bins[i]) / 2, y=20, text=label, font=dict(color='white', size=12), showarrow=False)

# Mise en forme du graphique
fig.update_layout(
    title='Distribution du Score Touristique',
    xaxis_title='Score Touristique',
    yaxis_title='Nombre d\'observations',
    bargap=0.05,
    showlegend=False,
    template='plotly_dark'
)

# Affichage du graphique interactif
fig.show()


# Appliquer la fonction à la colonne 'id_pays'
df_ecol['id_pays'] = df_ecol['id_pays'].apply(iso2_to_iso3)

df_ecol.head()

# Créer une carte du monde
fig = px.choropleth(df_ecol, locations='id_pays', color='label_ecol', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()

# Nombre de score enregistré : 151/196


max :  0.8742857199999999 min :  0.0018257330541103425 variance:  0.17147176859325916 esperance:  0.2559393077856318


## Score économique 

In [350]:
# Normalisation des indicateurs par la méthode min-max
df_eco['arriveesTotal'] = df_eco['arriveesTotal'] / df_eco['arriveesTotal'].max()
df_eco['pibParHab'] = df_eco['pibParHab'] / df_eco['pibParHab'].max()
df_eco['idh'] = df_eco['idh'] / df_eco['idh'].max()


# Les poids
poids_eco = {
    'arriveesTotal': 2,
    'pibParHab': 2,
    'idh' : 2
}

# Calcul du score économique 
df_eco['score_economique'] = df_eco.apply(lambda row: sum(float(row[var]) * poids_eco[var] for var in poids_eco) / sum(poids_eco.values()), axis=1)
df_eco.head()

max_score = df_eco['score_economique'].max()
min_score = df_eco['score_economique'].min()

variance = df_eco['score_economique'].std()
esperance = df_eco['score_economique'].mean()

# Les limites des scores/labels
bins = [min_score, 0.28, 0.42, 0.56, 0.7, max_score]

print("max : ",max_score, "min : ",min_score, "variance: ",variance, "esperance: ",esperance)


# Labels
labels = ['E', 'D', 'C', 'B', 'A']

# création d'une nouvelle colonne pour les labels
df_eco['label_eco'] = pd.cut(df_eco['score_economique'], bins=bins, labels=labels)

df_eco = df_eco.sort_values(by='score_economique', ascending=False)

df_eco.dropna(inplace=True)

# application de la fonction faite précédement 
df_eco['id_pays'] = df_eco['id_pays'].apply(iso2_to_iso3)

df_eco.head()

# Visualisation par une carte
fig = px.choropleth(df_eco, locations='id_pays', color='label_eco', hover_name='pays', projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")
fig.show()

# Nombre de score enregistré : 130 / 196 

max :  0.7938203729074744 min :  0.13791954268099327 variance:  0.13938543448940582 esperance:  0.35361106904067097


## Score découverte 

In [351]:

# Normalisation des indicateurs par la méthode min-max
df_decouverte['arriveesTotal'] = df_decouverte['arriveesTotal'] / df_decouverte['arriveesTotal'].max()
df_decouverte['GES_hab'] = df_decouverte['GES_hab'].min() / df_decouverte['GES_hab']
df_decouverte['idh'] = df_decouverte['idh'] / df_decouverte['idh'].max()


# Les poids pour le tourisme de découverte
poids_decouverte = {
    'arriveesTotal': 2,
    'idh' : 2,
    'GES_hab': 1
}

# Calcul du score pour le tourisme culturel et historique / tourisme d'exploration et de découverte
df_decouverte['score_decouverte'] = df_decouverte.apply(lambda row: sum(float(row[var]) * poids_decouverte[var] for var in poids_decouverte) / sum(poids_decouverte.values()), axis=1)

max_score = df_decouverte['score_decouverte'].max()
min_score = df_decouverte['score_decouverte'].min()
esperance = df_decouverte['score_decouverte'].mean()
variance = df_decouverte['score_decouverte'].std()


print("max : ",max_score, "min : ",min_score, "variance: ",variance, "esperance: ",esperance)

#Pour le score découverte
bins = [min_score, 0.27, 0.36, 0.45, 0.54, max_score]

# Les labels
labels = ['E', 'D', 'C', 'B', 'A']


# Création d'une nouvelle colonne pour les labels
df_decouverte['label_decouverte'] = pd.cut(df_decouverte['score_decouverte'], bins=bins, labels=labels)

df_decouverte = df_decouverte.sort_values(by='score_decouverte', ascending=False)

print(df_decouverte.head())

df_decouverte.dropna(inplace=True)


df_decouverte['id_pays'] = df_decouverte['id_pays'].apply(iso2_to_iso3)

df_decouverte.head()

# Visualisation par une carte
fig = px.choropleth(df_decouverte, locations='id_pays', color='label_decouverte', hover_name='pays', projection='natural earth')
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")
fig.show()

# 99 score sur 196 enregistré 

max :  0.7964632790021787 min :  0.17432017511517411 variance:  0.08742056217183063 esperance:  0.36583160610316195
              pays id_pays  annee  arriveesTotal   GES_hab       idh  \
47          France      FR   2020       1.000000  0.113158  0.934579   
69          Italie      IT   2020       0.604309  0.121773  0.926272   
43         Espagne      ES   2020       0.454203  0.108696  0.928349   
144  United States      US   2020       0.460896  0.033824  0.958463   
89         Mexique      MX   2020       0.595528  0.103231  0.786085   

     score_decouverte label_decouverte  
47           0.796463                A  
69           0.636587                A  
43           0.574760                A  
144          0.574509                A  
89           0.573292                A  
